In [36]:
import pandas as pd
import json

In [37]:
df = pd.read_csv('REBNY add_edit_ field requirements - Requirements.csv')
df.head(10)

,element_id,Virtual attribute type,Label,Required condition,Listing type
0,sale_or_rental,NaN,Sale or rental,TRUE,"Rental, Sale"
1,lease_type,NaN,Lease type,"{""and"": [\n {""=="" : [ { ""var"" : ""sale_or_rent...",Rental
2,building_id,NaN,Address,TRUE,"Rental, Sale"
3,property_type__sales,"listing, property_type_code",Property type,TRUE,"Rental, Sale"
4,structure_type,"feature, structure_type",Structure type,TRUE,"Rental, Sale"
5,units_attributes,NaN,Unit number,"{""some"": [\n [{""var"": ""property_type__sales""}...","Rental, Sale"
6,tax_block,"feature, tax_block",Tax block,TRUE,"Rental, Sale"
7,tax_lot,"feature, tax_lot",Tax lot,"\n {""!"": [\n [{""var"": ""property_type__sale...","Rental, Sale"
8,status_code,"listing, standard_status",Status,TRUE,"Rental, Sale"
9,contract_date,"listing, contract_date",Contract date,"{ ""or"" : [\n{""=="" : [ { ""var"" : ""status_code"" ...","Rental, Sale"


In [38]:
df_req = df[df['Required condition'] == 'TRUE']
df_cond = df[df['Required condition'] != 'TRUE']

In [39]:
len(df_req), len(df_cond)

(27, 62)

In [40]:
query = '''select listings.id, 
listings.brokerage_id, 
reso_reso_properties.listing_id, 
reso_reso_properties.unparsed_address,
reso_reso_properties.unit_number,
reso_reso_properties.list_agent_full_name,
reso_reso_properties.co_list_agent_full_name,
'''

In [41]:
query += '-- Required Fields\n'
for var, var1 in zip(df_req['element_id'], df_req['Virtual attribute type']):
    if pd.isna(var1):
        query += f'{var},\n'
    else:
        tmp = var1.split(',')
        query += f'{tmp[0]}s.{tmp[1].strip()},\n'
query += '-- Conditional Fields\n'
for var, var1 in zip(df_cond['element_id'], df_cond['Virtual attribute type']):
    if pd.isna(var1):
        query += f'{var},\n'
    else:
        tmp = var1.split(',')
        query += f'{tmp[0]}s.{tmp[1].strip()},\n'
print(query)

select listings.id, 
listings.brokerage_id, 
reso_reso_properties.listing_id, 
reso_reso_properties.unparsed_address,
reso_reso_properties.unit_number,
reso_reso_properties.list_agent_full_name,
reso_reso_properties.co_list_agent_full_name,
-- Required Fields
sale_or_rental,
building_id,
listings.property_type_code,
features.structure_type,
features.tax_block,
listings.standard_status,
list_date,
expiration,
lease_term_min_months,
listings.concessions_given,
listing_agreement,
listings.cobroke_agreement,
propertiess.agent_id,
office_id,
listings.buyer_brokerage_compensation_type,
description,
showing_instructions,
published,
features.year_built,
amenitys.garage,
features.elevators_total,
attendance_type ,
features.stories_total,
features.number_of_units_total,
features.laundry_features,
features.bedrooms,
rooms,
-- Conditional Fields
lease_type,
units_attributes,
features.tax_lot,
listings.contract_date,
sale_date,
sale_price,
listingss.buyer_agent_id,
listings.cancellation_date,
furni

In [42]:
for r in df_cond['Required condition']:
    print(r)

{"and": [
  {"==" : [ { "var" : "sale_or_rental" }, "R"]},
  {"!": [
    [{"var": "property_type__sales"}],
    {"in": [ {"var": ""}, [400, "400"] ]}
  ]}
]}
{"some": [
  [{"var": "property_type__sales"}],
  {"in": [ {"var": ""}, [400, "400",  530, "530", 540, "540", 550, "550",  570, "570", 580, "580"] ]}
]}

  {"!": [
    [{"var": "property_type__sales"}],
    {"in": [ {"var": ""}, [100, "100", 400, "400"] ]}
  ]}
{ "or" : [
{"==" : [ { "var" : "status_code" }, "240"]},
{"==" : [ { "var" : "status_code" }, "400" ] },
{"==" : [ { "var" : "status_code" }, "500"]}
] }
{ "or" : [
{"==" : [ { "var" : "status_code" }, "400" ] },
{"==" : [ { "var" : "status_code" }, "500"]}
] }
{ "or" : [
{"==" : [ { "var" : "status_code" }, "400" ] },
{"==" : [ { "var" : "status_code" }, "500"]}
] }
{ "or" : [
{"==" : [ { "var" : "status_code" }, "400" ] },
{"==" : [ { "var" : "status_code" }, "500"]}
] }
{"==": [ { "var" : "status_code" }, "620" ]}
{"and": [
  {"==" : [ { "var" : "sale_or_rental" }, "R"]}

In [43]:
def load_json(s):
    try:
        return json.loads(s)
    except:
        return json.loads(s[:s.find('Conditional:')])

In [44]:
elements = []
'''for i in x:
    if i == 'var':
        elements.append(x[i])
    for j in x[i]:
        print(j)
        if j == 'var':
            elements.append(x[i][j])'''

"for i in x:\n    if i == 'var':\n        elements.append(x[i])\n    for j in x[i]:\n        print(j)\n        if j == 'var':\n            elements.append(x[i][j])"

In [45]:
y = df_cond['Required condition'][1]

In [46]:
y.split(':')

['{"and"',
 ' [\n  {"==" ',
 ' [ { "var" ',
 ' "sale_or_rental" }, "R"]},\n  {"!"',
 ' [\n    [{"var"',
 ' "property_type__sales"}],\n    {"in"',
 ' [ {"var"',
 ' ""}, [400, "400"] ]}\n  ]}\n]}']

In [47]:
switch = False
elements = []
for y in df_cond['Required condition']:
    for x in y.split(':'):
        if switch:
            elements.append(x[x.find('"')+1:x[x.find('"')+1:].find('"')+2])
            switch = False
        if 'var' in x:
            switch = True

In [48]:
set(elements)

{'',
 'basement_y_n',
 'bonus_y_n',
 'buyer_brokerage_compensation_type',
 'commission_type',
 'concessions_given',
 'configuration',
 'cooling_y_n',
 'flip_tax_amount',
 'flip_tax_pct',
 'flip_tax_type__form_sections',
 'furnished',
 'garage_spaces_assigned_y_n',
 'heating_y_n',
 'listing_agreement',
 'new_construction_y_n',
 'new_development_y_n',
 'owner_pays_plus_concessions_y_n',
 'pet_policy',
 'property_type__sales',
 'published',
 'rls',
 'sale_or_rental',
 'showing_start_time',
 'status_code',
 'tax_abatement_y_n__form_sections',
 'total_fireplaces',
 'view_y_n'}

In [49]:
from json_logic import jsonLogic

In [50]:
df_cond['rules'] = [x.replace('\n', '').replace('property_type__sales','property_type_code').replace('status_code', 'standard_status').replace('50"', 'Coming Soon"').replace('100', 'Active').replace('240', 'Pending').replace('configuration', 'configuration_type') for x in df_cond['Required condition']]

/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_65239/4265772475.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cond['rules'] = [x.replace('\n', '').replace('property_type__sales','property_type_code').replace('status_code', 'standard_status').replace('50"', 'Coming Soon"').replace('100', 'Active').replace('240', 'Pending').replace('configuration', 'configuration_type') for x in df_cond['Required condition']]


In [51]:
json.loads(df_cond['rules'][10])

{'or': [{'==': [{'var': 'standard_status'}, '400']},
  {'==': [{'var': 'standard_status'}, '500']}]}

In [52]:
df_req.head()

,element_id,Virtual attribute type,Label,Required condition,Listing type
0,sale_or_rental,NaN,Sale or rental,TRUE,"Rental, Sale"
2,building_id,NaN,Address,TRUE,"Rental, Sale"
3,property_type__sales,"listing, property_type_code",Property type,TRUE,"Rental, Sale"
4,structure_type,"feature, structure_type",Structure type,TRUE,"Rental, Sale"
6,tax_block,"feature, tax_block",Tax block,TRUE,"Rental, Sale"


In [53]:
def get_name(element_id, virtual_id):
    if pd.isna(virtual_id):
        return element_id
    else:
        return virtual_id.split(',')[1].strip()

In [54]:
df_req['field_name'] = [get_name(x,y) for x,y in zip(df_req['element_id'], df_req['Virtual attribute type'])]
df_cond['field_name'] = [get_name(x,y) for x,y in zip(df_cond['element_id'], df_cond['Virtual attribute type'])]

/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_65239/643459082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_req['field_name'] = [get_name(x,y) for x,y in zip(df_req['element_id'], df_req['Virtual attribute type'])]
/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_65239/643459082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cond['field_name'] = [get_name(x,y) for x,y in zip(df_cond['element_id'], df_cond['Virtual attribute type'])]


In [55]:
listings_df = pd.read_csv('listings1.csv')

In [66]:
df_req.head()

,element_id,Virtual attribute type,Label,Required condition,Listing type,field_name
0,sale_or_rental,NaN,Sale or rental,TRUE,"Rental, Sale",sale_or_rental
2,building_id,NaN,Address,TRUE,"Rental, Sale",building_id
3,property_type__sales,"listing, property_type_code",Property type,TRUE,"Rental, Sale",property_type_code
4,structure_type,"feature, structure_type",Structure type,TRUE,"Rental, Sale",structure_type
6,tax_block,"feature, tax_block",Tax block,TRUE,"Rental, Sale",tax_block


In [67]:
keys = []
for index, row in listings_df.iterrows():
    key = ''
    for i, r in df_req.iterrows():
        if pd.isna(row[r['field_name'].strip()]) and row['sale_or_rental'] in r['Listing type']:
            key += r['Label']
            key += '|'
    
    if row.sale_or_rental == 'R' and row.property_type_code == 400 and pd.isna(row.lease_type):
        key += 'Lease type|'
    #elif row.property_type_code in [400, 530, 540, 550, 570, 580] and pd.isna(row.units_attributes):
     #   key += 'units_attributes|'
    if row.property_type_code not in [100, 400] and pd.isna(row.tax_lot):
        key += 'Tax Lot|'
    if row.standard_status in ["Pending", "Sold"] and pd.isna(row.contract_date):
        key += 'Contract Date|'
    if row.standard_status == "Sold" and pd.isna(row.close_date):
        key += 'Close Date|'
    if row.standard_status == "Sold" and pd.isna(row.close_price):
        key += 'Close Price|'
    if row.standard_status == "Sold" and pd.isna(row.buyer_agent_id):
        key += 'Buyer agents|'
    if row.standard_status == "Cancelled" and pd.isna(row.cancellation_date):
        key += 'Cancellation Date|'
    if row.sale_or_rental == 'R' and row.property_type_code not in [530,"530",540,"540",550,"550",560,"560",580,"580",570,"570",590,"590"] and pd.isna(row.furnished):
        key += 'furnished'
    if row.sale_or_rental == 'S' and row.property_type_code in [530, 540, 550] and pd.isna(row.special_listing_conditions):
        key += 'Special listing conditions|'
    if pd.isna(row.list_price):
        key += 'List price|'
    #elif row.sale_or_rental == 'R' and pd.isna(row.rental_price):
     #   key += 'rental_price|'
    if row.sale_or_rental == 'R' and row.furnished in ['furnished', 'partially', 'negotiable'] and pd.isna(row.furnished_rent):
        key += 'Furnished rental price|'
    if row.sale_or_rental == 'R' and row.property_type_code in [530,"530",540,"540",550,"550",560,"560",580,"580",570,"570",590,"590"] and pd.isna(row.available_date):
        key += 'Availability date|'
    if row.furnished in ['Yes', 'Optional'] and row.property_type_code not in [100, 400] and row.sale_or_rental == 'R' and pd.isna(row.lease_term_max_months):
        key += 'Maximum lease months|'
    if row.property_type_code in [530, "530",  540, "540", 550, "550"] and row.sale_or_rental == 'S' and pd.isna(row.max_financing_pct):
        key += 'Max financing (%)|'
    if row.property_type_code in [530, "530",  540, "540", 550, "550"] and row.sale_or_rental == 'S' and pd.isna(row.maximum_financing_remarks):
        key += 'Max financing remarks|'
    if row.property_type_code in [530, "530", 550, "550"] and row.sale_or_rental == 'S' and pd.isna(row.number_of_shares):
        key += 'Number of shares|'
    if row.property_type_code in [540, "540"] and row.sale_or_rental == 'S' and pd.isna(row.percent_of_common_elements):
        key += 'Percent of common elements|'
    if row.property_type_code in [530, "530",  540, "540", 550, "550"] and row.sale_or_rental == 'S' and pd.isna(row.hoa_fee):
        key += 'Monthlies/recurring fees|'
    if row.property_type_code in [530, "530",  540, "540", 550, "550"] and row.sale_or_rental == 'S' and pd.isna(row.association_fee_frequency):
        key += 'Fee frequency|'
    if row.property_type_code == 540 and row.sale_or_rental == 'S' and pd.isna(row.real_estate_tax):
        key += 'Fee frequency|'
    if row.property_type_code in [530, "530",  540, "540", 550, "550"] and row.sale_or_rental == 'S' and (pd.isna(row.flip_tax_pct) and pd.isna(row.flip_tax_amount)):
        key += 'Flip tax amount ($)|Flip tax (%)'
    if row.flip_tax_amount > 0 or row.flip_tax_pct > 0 and pd.isna(row.flip_tax_description):
        key += 'Flip tax remarks|'
    if row.concessions_given == 'yes' and pd.isna(row.concessions_comments):
        key += 'Concessions comments|'
    if row.concessions_given == 'yes' and row.commission_type == 'Owner Pays' and row.sale_or_rental == 'R' and pd.isna(row.owner_pays_plus_concessions_y_n):
        key += 'Owner pays commission + concessions?|'
    if (row.owner_pays_plus_concessions_y_n == True or row.commission_type == 'Owner Pays') and row.sale_or_rental == 'R' and pd.isna(row.owner_pays):
        key += 'Owner pays|'
    if (row.owner_pays_plus_concessions_y_n == True or row.commission_type == 'Owner Pays') and row.sale_or_rental == 'R' and pd.isna(row.owner_pays_remarks):
        key += 'Owner pays remarks|'
    if row.sale_or_rental == 'R' and row.property_type_code != 100 and pd.isna(row.commission_type):
        key += 'Rental compensation type|'
    #elif row.buyer_brokerage_compensation_type == 'percent' and pd.isna(row.commission_amount_percentage):
     #   key += 'commission_amount_percentage|'
    #elif row.buyer_brokerage_compensation_type == 'dollar' and pd.isna(row.commission_amount_dollar):
     #   key += 'commission_amount_dollar|'
    if row.commission_type == 'Co-broke' and pd.isna(row.commission_remarks):
        key += 'Commission remarks|'
    if row.bonus_y_n == True and row.sale_or_rental == 'S' and pd.isna(row.bonus):
        key += 'Bonus remarks|'
    if (not pd.isna(row.showing_start_time)) and pd.isna(row.showing_end_time):
        key += 'End time|'
    if row.published == True and row.property_type_code != 100 and pd.isna(row.syndicate_to):
        key += 'REBNY syndication portals|'
    if row.property_type_code not in [400, 100] and pd.isna(row.overall_condition):
        key += 'Property condition|'
    if row.new_development_y_n == True or row.new_construction_y_n == True and pd.isna(row.sponsored):
        key += 'Resale/sponsor|'
    if row.property_type_code in [560, 580, 570, 590] and pd.isna(row.lot_size_area):
        key += 'Lot size area|'
    if row.property_type_code in [560, 580, 570, 590] and pd.isna(row.lot_size_units):
        key += 'Lot size units|'
    if row.property_type_code in [560, 580, 570, 590] and pd.isna(row.lot_size_dimensions):
        key += 'Lot size dimensions|'
    if row.property_type_code in [560, 580, 570, 590] and pd.isna(row.building_size_dimensions):
        key += 'Building dimensions|'
    if (row.property_type_code in [560, "560",  570, "570",  580, "580",   590, "590"] or row.configuration_type in ['duplex', 'triplex']) and pd.isna(row.building_area_total):
        key += 'Building area total|'
    if (row.property_type_code in [560, "560",  570, "570",  580, "580",   590, "590"] or row.configuration_type in ['duplex', 'triplex']) and pd.isna(row.building_area_units):
        key += 'Building area units|'
    if row.property_type_code == 540 and pd.isna(row.lot_size_square_feet):
        key += 'Living area (sq ft)|'
    if row.garage == True and pd.isna(row.garage_spaces):
        key += 'Number of assigned spaces|'
    if row.property_type_code != 100 and pd.isna(row.pet_policy):
        key += 'Pets allowed|'
    if ('unit_size_limit' in str(row.pet_policy) or 'unit_number_limit' in str(row.pet_policy) or 'building_size_limit' in str(row.pet_policy) or 'building_number_limit' in str(row.pet_policy) or 'breed_restrictions' in str(row.pet_policy)) and pd.isna(row.pet_policy_remarks):
        key += 'Pets allowed comments|'
    if row.basement_y_n ==  True and pd.isna(row.basement_type):
        key += 'Basement|'
    if row.heating_y_n ==  True and pd.isna(row.heating):
        key += 'Heating features|'
    if row.cooling_y_n ==  True and pd.isna(row.cooling):
        key += 'Cooling features|'
    if row.view_y_n == True and pd.isna(row.view):
        key += 'view|'
    if row.total_fireplaces > 0 and pd.isna(row.fireplace_features):
        key += 'Fireplace features|'
    if (row.property_type_code != 100 or row.property_type_code != 400) and pd.isna(row.bathrooms):
        key += 'Full bathrooms|'
    if (row.property_type_code != 100 or row.property_type_code != 400) and pd.isna(row.half_bathrooms):
        key += 'Half bathrooms|'
    keys.append(key)

In [57]:
keys

['lease_term_min_months|',
 'lease_term_min_months|elevators_total|stories_total|number_of_units_total|laundry_features|commission_remarks|pet_policy|',
 'lease_term_min_months|elevators_total|stories_total|number_of_units_total|laundry_features|commission_remarks|overall_condition|garage_spaces|pet_policy|',
 'lease_term_min_months|elevators_total|stories_total|number_of_units_total|laundry_features|commission_remarks|overall_condition|pet_policy|',
 'lease_term_min_months|',
 '',
 'elevators_total|stories_total|number_of_units_total|max_financing_pct|flip_tax_amount|flip_tax_pctoverall_condition|',
 'elevators_total|stories_total|number_of_units_total|laundry_features|overall_condition|lot_size_area|lot_size_units|lot_size_dimensions|building_size_dimensions|building_area_total|building_area_units|pet_policy|',
 '',
 'agent_id|lease_type|',
 '',
 'lease_term_max_months|',
 'elevators_total|stories_total|number_of_units_total|laundry_features|number_of_shares|flip_tax_description|over

In [58]:
df_cond[df_cond['element_id'] == 'contract_date']['rules']

9    { "or" : [{"==" : [ { "var" : "standard_status...
Name: rules, dtype: object

In [59]:
jsonLogic(json.loads(df_cond[df_cond['element_id'] == 'contract_date']['rules'].values[0]), listings_df[listings_df.index == 17].to_json(orient='records'))

False

In [60]:
listings_df[listings_df.index == 17].to_json(orient='records')

'[{"id":7714874,"brokerage_id":1886,"listing_id":"PRCH-7714874","unparsed_address":"791 Park Avenue","unit_number":"14B","list_agent_full_name":"Lisa Chajet","co_list_agent_full_name":"Bonnie Chajet","agent_id":3157.0,"sale_or_rental":"S","building_id":22164,"property_type_code":550,"structure_type":"{\'none\': True}","tax_block":1408.0,"standard_status":"Active","list_date":"26\\/09\\/23 17:04","expiration":"24\\/03\\/24 04:00","lease_term_min_months":null,"concessions_given":"call_listing_agent","listing_agreement":"exclusive_agency","cobroke_agreement":"REBNY Universal Co-broke","office_id":8021.0,"buyer_brokerage_compensation_type":"percent","description":"791 Park Avenue, 14th Floor \\n \\nOn the 14th Floor of 791 Park Avenue sits a highly coveted 12-room, pre-war masterpiece.\\n \\nEnter Apartment 14B from a private elevator landing that opens into a magnificent entrance gallery serving as the focal point of the home. It connects seamlessly to the living room\\u2014an open expans

In [61]:
df_cond['rules'].values[2].strip()

'{"!": [    [{"var": "property_type_code"}],    {"in": [ {"var": ""}, [Active, "Active", 400, "400"] ]}  ]}'

In [62]:
json.loads(df_cond['rules'].values[2].replace('[ {"var": ""},', ''))

JSONDecodeError: Expecting value: line 1 column 59 (char 58)

In [ ]:
listings_df[listings_df['cooling_y_n'] == True]

,id,brokerage_id,listing_id,unparsed_address,unit_number,list_agent_full_name,co_list_agent_full_name,agent_id,sale_or_rental,building_id,...,half_bathrooms,basement_y_n,configuration_type,cooling_y_n,garage_spaces_assigned_y_n,new_construction_y_n,new_development_y_n,heating_y_n,showing_start_time,total_fireplaces
413,7742397,1824,PRCH-7742397,210 East 47th Street,9D,Jeff Brenner,NaN,7581.0,R,41280,...,0.0,NaN,NaN,True,False,NaN,NaN,NaN,NaN,NaN
417,7745104,1824,PRCH-7745104,343 Prospect Avenue,NaN,Karen Heyman,Alan Heyman,2213.0,S,59157,...,2.0,False,NaN,True,False,NaN,False,NaN,NaN,NaN
418,7746935,1824,PRCH-7746935,112 West 72nd Street,PHA,Kathryn Swift,Cyrus Greenspon,7945.0,R,10004,...,0.0,NaN,NaN,True,False,NaN,NaN,NaN,NaN,NaN
420,7747027,1824,PRCH-7747027,244 Madison Avenue,UPH,Glenn Norrgard,NaN,8511.0,S,16869,...,0.0,NaN,NaN,True,False,NaN,False,NaN,NaN,NaN
1689,7588865,1886,PRCH-7588865,400 East 56th Street,3B,Andrea E. Wernick,Rashi Malhotra,4468.0,S,28138,...,0.0,False,NaN,True,False,NaN,False,True,NaN,NaN


In [68]:
listings_df['keys'] = keys

In [69]:
l = listings_df[listings_df['keys'] != ''][['brokerage_id',	'listing_id','unparsed_address', 'unit_number',	'list_agent_full_name',	'co_list_agent_full_name',	'sale_or_rental',	'standard_status',	'list_date','expiration','bedrooms', 'rooms','contract_date','sale_date','sale_price', 'cancellation_date', 'list_price', 'keys']]
len(l)

1684

In [75]:
agents1 = set(l[l['brokerage_id'] == 1886]['list_agent_full_name'])
agents2 = set(l[l['brokerage_id'] == 1824]['list_agent_full_name'])
l[(l['list_agent_full_name'].isin(agents1)) | (l['co_list_agent_full_name'].isin(agents1))].to_excel('data/Warburg/master_missing_fields.xlsx')
l[(l['list_agent_full_name'].isin(agents2)) | (l['co_list_agent_full_name'].isin(agents2))].to_excel('data/Sothebys/master_missing_fields.xlsx')


for agent in agents1:
    l[(l['list_agent_full_name'] == agent) | (l['co_list_agent_full_name'] == agent)].to_csv(f'data/Warburg/{agent}.csv', index=False)


for agent in agents2:
    l[(l['list_agent_full_name'] == agent) | (l['co_list_agent_full_name'] == agent)].to_csv(f'data/Sothebys/{agent}.csv', index=False)

In [72]:
agents2

{'Aimee Denaro Becker',
 'Alain Azaria',
 'Alexander Brown',
 'Alexander Novack',
 'Alexandre  Ragovski',
 'Alison Bartlett',
 'Alison Glestein',
 'Allen Wu',
 'Allison B. Koffman',
 'Amanda Field Jordan',
 'Amanda Goldworm',
 'Amy Lee',
 'Andrew Borland',
 'Andrew Sideras',
 'Angela  Wu',
 'Annaliese Kirby',
 'Anne Aransaenz',
 'Anouk Nora',
 'Anthony Ligorelli',
 'Armin Allen',
 'Barbara Helms',
 'Barbara Ireland',
 'Benjamin Pofcher',
 'Brad Ingalls',
 'Brenda Stauffer Powers',
 'Brigitte Goldenberg',
 'Bryan Baltimore',
 'Bryan Kreider',
 'Bryan Lawson',
 'Caroline E. Y. Guthrie',
 'Cathy Taub',
 'Celeste Pandhi',
 'Cherie Hinson',
 'Chris Casey',
 'Chris Poore',
 'Christian Bindel',
 'Claire Groome',
 'Colin Montgomery',
 'Cortnee B. Glasser',
 'Craig George',
 'Cybele Kadagian',
 'Dana Kirshenbaum',
 'Dana Williams',
 'Daniel Chen',
 'Danielle Wiedemann',
 'David Hsu',
 'Debra Pinto',
 'Diana Devendorf Rice',
 'Dianne M. Weston',
 'Don Hurwitz',
 'Donald Cantasano',
 'Douglas Tra

In [74]:
agents1

{'Adjina Dekidjiev',
 'Alan Levy',
 'Alan Louie',
 'Amanda P. Jhones',
 'Amber L. Roush',
 'Andrea E. Wernick',
 'Annie Cion Gruenberger',
 'Arlene S. Reed',
 'Becki Danchik',
 'Bonnie Chajet',
 'Bonnie Lindenbaum',
 'Cecilia Serrano',
 'Christopher Totaro',
 'David Harris',
 'Donald Toresco',
 'Dorothy Schrager',
 'Ellen I Sykes',
 'Gerard Splendore',
 'Harriet Kaufman',
 'Jacqueline L. Kurtz',
 'James Harrison',
 'Jane Katz',
 'Jane R. Andrews',
 'Janet D. Traynor',
 'Jeanne Byers',
 'Jeremy Kamm',
 'Judith B. Thorn',
 'Judy Kloner',
 'June Gottlieb',
 'Karen Kostiw',
 'Kate Wollman-Mahan',
 'Kwesi O. Kwarteng',
 'Linda Reiner',
 'Lisa Camillieri',
 'Lisa Chajet',
 'Lisa Tarnopol Deslauriers',
 'Maria E. Daou',
 'Maria Kourepenos',
 'Marvin Dudley',
 'Mary Hall Mayer',
 'Michael C. Weiner',
 'Mihal Gartenberg',
 'Monisha S. Rana',
 'Parisa M. Afkhami',
 'Rachel Ostow Lustbader',
 'Rebecca Blacker',
 'Regina Weis',
 'Robert A. Schulman',
 'Ronnie G. Lane',
 'Samantha Rose Frith',
 'Sh